In [45]:
from models.cswin import CSWin_64_24322_small_224
import torch
import torch.nn as nn
import os
from collections import OrderedDict
import logging
_logger = logging.getLogger('train')

In [46]:
def load_checkpoint(args, model, checkpoint_path, use_ema=False):
    if checkpoint_path and os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        state_dict_key = 'state_dict'
        model_key = 'model'
        if isinstance(checkpoint, dict):
            if use_ema and 'state_dict_ema' in checkpoint:
                state_dict_key = 'state_dict_ema'
        if state_dict_key and state_dict_key in checkpoint:
            new_state_dict = OrderedDict()
            for k, v in checkpoint[state_dict_key].items():
                # strip `module.` prefix
                name = k[7:] if k.startswith('module') else k
                if False and 'head' in k:
                    continue
                new_state_dict[name] = v
            state_dict = new_state_dict
        elif model_key and model_key in checkpoint:
            new_state_dict = OrderedDict()
            for k, v in checkpoint[model_key].items():
                # strip `module.` prefix
                name = k[7:] if k.startswith('module') else k
                if False and 'head' in k:
                    continue
                new_state_dict[name] = v
            state_dict = new_state_dict

        else:
            state_dict = checkpoint
        if True == 0:
            _logger.info("Loaded {} from checkpoint '{}'".format(state_dict_key, checkpoint_path))
    else:
        if True :
            _logger.error("No checkpoint found at '{}'".format(checkpoint_path))
        raise FileNotFoundError()
    model_dict = model.state_dict()
    pretrained_dict = state_dict
    loaded_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    load_dic = [k for k, v in pretrained_dict.items() if k in model_dict]
    miss_dic = [k for k, v in pretrained_dict.items() if not (k in model_dict)]
    unexpect_dic = [k for k, v in model_dict.items() if not (k in pretrained_dict)]
    if True :
        print ("Miss Keys:", miss_dic)
        print ("Ubexpected Keys:", unexpect_dic)
    model_dict.update(loaded_dict)
    model.load_state_dict(model_dict, strict=True)

In [47]:
model1=CSWin_64_24322_small_224()

In [48]:
load_checkpoint(None,model1,"/home/dc/ssd2/vscode20240117/CSWin-Transformer-main/cswin_small_224.pth",True)

Miss Keys: []
Ubexpected Keys: []


In [49]:
model2=torch.load("cswin_small_224.pth")

In [50]:
for key, value in model2["state_dict_ema"].items():
    print(key,value.size(),sep="  ")

stage1_conv_embed.0.weight  torch.Size([64, 3, 7, 7])
stage1_conv_embed.0.bias  torch.Size([64])
stage1_conv_embed.2.weight  torch.Size([64])
stage1_conv_embed.2.bias  torch.Size([64])
stage1.0.qkv.weight  torch.Size([192, 64])
stage1.0.qkv.bias  torch.Size([192])
stage1.0.norm1.weight  torch.Size([64])
stage1.0.norm1.bias  torch.Size([64])
stage1.0.proj.weight  torch.Size([64, 64])
stage1.0.proj.bias  torch.Size([64])
stage1.0.attns.0.get_v.weight  torch.Size([32, 1, 3, 3])
stage1.0.attns.0.get_v.bias  torch.Size([32])
stage1.0.attns.1.get_v.weight  torch.Size([32, 1, 3, 3])
stage1.0.attns.1.get_v.bias  torch.Size([32])
stage1.0.mlp.fc1.weight  torch.Size([256, 64])
stage1.0.mlp.fc1.bias  torch.Size([256])
stage1.0.mlp.fc2.weight  torch.Size([64, 256])
stage1.0.mlp.fc2.bias  torch.Size([64])
stage1.0.norm2.weight  torch.Size([64])
stage1.0.norm2.bias  torch.Size([64])
stage1.1.qkv.weight  torch.Size([192, 64])
stage1.1.qkv.bias  torch.Size([192])
stage1.1.norm1.weight  torch.Size([64]

In [51]:
# model1.load_state_dict(model2)